In [1]:
import pandas as pd
from simpletransformers.classification import ClassificationModel
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
import os
import torch

In [2]:
kfold = 10
train_folds_df = pd.read_csv("severityData_cleaned_deneme.csv", sep="|", encoding="utf-8")
train_folds_df.head()

,tweet,severity
0,ulan wifiye baglıyım ben baglıyken turkcell in...,1
1,dk gb internet mb kasar turkcell duser oc cocu...,1
2,turkcell superonline reklamı kötü bir reklam g...,0
3,turkcell pahalı,0
4,turkcell kasta internetin cekmiyor,2


In [3]:
if 'kfold' not in train_folds_df.columns:
    kf = KFold(n_splits=kfold, shuffle=True, random_state=42)
    train_folds_df['kfold'] = -1
    for fold, (train_index, val_index) in enumerate(kf.split(train_folds_df)):
        train_folds_df.loc[val_index, 'kfold'] = fold

In [4]:
train_folds_df.head()

,tweet,severity,kfold
0,ulan wifiye baglıyım ben baglıyken turkcell in...,1,2
1,dk gb internet mb kasar turkcell duser oc cocu...,1,7
2,turkcell superonline reklamı kötü bir reklam g...,0,5
3,turkcell pahalı,0,1
4,turkcell kasta internetin cekmiyor,2,6


In [5]:
for fold in range(kfold):
    # Split data into train and validation sets
    train_df = train_folds_df[train_folds_df['kfold'] != fold]
    validation_df = train_folds_df[train_folds_df['kfold'] == fold]

    # Prepare the data for simpletransformers
    train_df = pd.DataFrame({
        'text': train_df['tweet'],  # Replace 'your_text_column' with the actual column name for text
        'labels': train_df['severity']
    })
    validation_df = pd.DataFrame({
        'text': validation_df['tweet'],  # Replace 'your_text_column' with the actual column name for text
        'labels': validation_df['severity']
    })

    # Initialize the ClassificationModel
    model = ClassificationModel('bert',
                                'dbmdz/bert-base-turkish-uncased',
                                num_labels=3,
                                use_cuda=True,  # Use CUDA
                                args={'reprocess_input_data': True,
                                      'overwrite_output_dir': True,
                                      'num_train_epochs': 15,
                                      'train_batch_size': 64,
                                      'fp16': False,
                                      'save_model_every_epoch': True,
                                      'save_eval_checkpoints': False,
                                      'output_dir': f'bert_model_fold_merge_{fold}',
                                      'save_steps': 0 })

/home/ensar/anaconda3/lib/python3.9/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at dbmdz/bert-base-turkish-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of the model checkpoint at dbmdz/bert-base-turkish-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTrai

In [6]:
model.train_model(train_df)

  0%|          | 0/18637 [00:00<?, ?it/s]

Epoch:   0%|          | 0/15 [00:00<?, ?it/s]

Running Epoch 0 of 15:   0%|          | 0/292 [00:00<?, ?it/s]

Running Epoch 1 of 15:   0%|          | 0/292 [00:00<?, ?it/s]

Running Epoch 2 of 15:   0%|          | 0/292 [00:00<?, ?it/s]

...

Running Epoch 11 of 15:   0%|          | 0/292 [00:00<?, ?it/s]

Running Epoch 12 of 15:   0%|          | 0/292 [00:00<?, ?it/s]

Running Epoch 13 of 15:   0%|          | 0/292 [00:00<?, ?it/s]

Running Epoch 14 of 15:   0%|          | 0/292 [00:00<?, ?it/s]

(4380, 0.09976890785399843)

In [7]:
result, model_outputs, wrong_predictions = model.eval_model(validation_df)

  0%|          | 0/2070 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/259 [00:00<?, ?it/s]

In [8]:
predictions = model_outputs.argmax(axis=1)
actuals = validation_df['labels'].values

# Print classification report
print(metrics.classification_report(actuals, predictions, digits=3))

              precision    recall  f1-score   support

           0      0.882     0.908     0.895      1054
           1      0.885     0.854     0.869       925
           2      0.870     0.879     0.874        91

    accuracy                          0.883      2070
   macro avg      0.879     0.880     0.879      2070
weighted avg      0.883     0.883     0.882      2070

